#### Task 2 Model Building and Training 


In [ ]:
# Import python library
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler,LabelEncoder
import os,sys

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
import mlflow
import mlflow.sklearn
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, LSTM, Dropout, Flatten

In [2]:
# Get the absolute path of the parent directory
rpath = os.path.abspath('..')
if rpath not in sys.path:
    sys.path.insert(0, rpath)

In [3]:
# import util functions
from scripts.utils import  *

In [4]:
# Load the datasets
fraud_data = pd.read_csv("../data/Fraud_Data.csv")
credit_data = pd.read_csv("../data/creditcard.csv")

In [5]:
# Credit data Information
credit_data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [6]:
# Fraud data information
fraud_data.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1
3,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0
4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0


In [7]:
# Set up MLflow
mlflow.set_experiment("Fraud Detection")

<Experiment: artifact_location='file:///home/brook/Music/10Academy/week-8/Adey-Innovations/notebooks/mlruns/105863702443849380', creation_time=1730276744241, experiment_id='105863702443849380', last_update_time=1730276744241, lifecycle_stage='active', name='Fraud Detection', tags={}>

Convert datetime strings to datetime objects


In [8]:
# Convert datetime strings to datetime objects
fraud_data['signup_time'] = pd.to_datetime(fraud_data['signup_time'])
fraud_data['purchase_time'] = pd.to_datetime(fraud_data['purchase_time'])

In [9]:
# Extract useful datetime components
fraud_data['signup_hour'] = fraud_data['signup_time'].dt.hour
fraud_data['signup_day'] = fraud_data['signup_time'].dt.dayofweek
fraud_data['purchase_hour'] = fraud_data['purchase_time'].dt.hour
fraud_data['purchase_day'] = fraud_data['purchase_time'].dt.dayofweek

# Drop the original datetime columns
fraud_data = fraud_data.drop(columns=['signup_time', 'purchase_time'])

Model training

Preprocess Fraud Data

In [10]:
# Preprocess Fraud Data
X_fraud = fraud_data.drop(columns=['class'])
y_fraud = fraud_data['class']

Preprocess Credit-card Data

In [11]:
# Preprocess creditcard.csv
X_credit = credit_data.drop(columns=['Class'])
y_credit = credit_data['Class']

Train-test split

In [12]:
# Train-test split
X_train_fraud, X_test_fraud, y_train_fraud, y_test_fraud = train_test_split(X_fraud, y_fraud, test_size=0.2, random_state=42, stratify=y_fraud)
X_train_cc, X_test_cc, y_train_cc, y_test_cc = train_test_split(X_credit, y_credit, test_size=0.2, random_state=42, stratify=y_credit)

In [13]:
# Convert datetime columns if any
if 'TransactionDate' in X_train_fraud.columns:  # Replace with your datetime column
    X_train_fraud['TransactionDate'] = pd.to_datetime(X_train_fraud['TransactionDate'])
    X_train_fraud['TransactionYear'] = X_train_fraud['TransactionDate'].dt.year
    X_train_fraud['TransactionMonth'] = X_train_fraud['TransactionDate'].dt.month
    X_train_fraud['TransactionDay'] = X_train_fraud['TransactionDate'].dt.day
    X_train_fraud['TransactionHour'] = X_train_fraud['TransactionDate'].dt.hour
    # Drop the original datetime column
    X_train_fraud = X_train_fraud.drop('TransactionDate', axis=1)

# Repeat for X_test_fraud if needed

# Select only numeric columns for scaling
numeric_cols_fraud = X_train_fraud.select_dtypes(include=['float64', 'int64']).columns
numeric_cols_cc = X_train_cc.select_dtypes(include=['float64', 'int64']).columns

Feature scaling (standardization)

In [14]:
# Feature scaling (standardization)
scaler = StandardScaler()

X_train_fraud_scaled = scaler.fit_transform(X_train_fraud[numeric_cols_fraud])
X_test_fraud_scaled = scaler.transform(X_test_fraud[numeric_cols_fraud])

X_train_cc_scaled = scaler.fit_transform(X_train_cc[numeric_cols_cc])
X_test_cc_scaled = scaler.transform(X_test_cc[numeric_cols_cc])

Evaluation Function

In [15]:
# Define Evaluation Function
def evaluate_model(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, zero_division=1)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    roc_auc = roc_auc_score(y_true, y_pred)
    return accuracy, precision, recall, f1, roc_auc

Log function

In [16]:
def log_metrics(model_name, accuracy, precision, recall, f1, roc_auc):
    mlflow.log_param("Model", model_name)
    mlflow.log_metric("Accuracy", accuracy)
    mlflow.log_metric("Precision", precision)
    mlflow.log_metric("Recall", recall)
    mlflow.log_metric("F1 Score", f1)
    mlflow.log_metric("ROC-AUC", roc_auc)

Train and Evaluate Models

In [17]:
# Train and Evaluate Models
def train_and_evaluate(model, X_train, X_test, y_train, y_test, model_name):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy, precision, recall, f1, roc_auc = evaluate_model(y_test, y_pred)
    
    with mlflow.start_run():
        mlflow.sklearn.log_model(model, model_name)
        log_metrics(model_name, accuracy, precision, recall, f1, roc_auc)
        
    print(f"{model_name} - Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1: {f1}, ROC-AUC: {roc_auc}")

In [18]:
# Initialize models
models = {
    'Logistic Regression': LogisticRegression(),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(n_estimators=100),
    'Gradient Boosting': GradientBoostingClassifier(),
    'MLP': MLPClassifier(max_iter=500)
}

In [19]:
from sklearn.preprocessing import LabelEncoder

# Label encode 'sex' column
label_encoder = LabelEncoder()
X_train_fraud['sex'] = label_encoder.fit_transform(X_train_fraud['sex'])
X_test_fraud['sex'] = label_encoder.transform(X_test_fraud['sex'])

In [20]:
# Frequency encoding for 'device_id', 'source', and 'browser'
for col in ['device_id', 'source', 'browser']:
    freq_encoding = X_train_fraud[col].value_counts(normalize=True)
    X_train_fraud[col] = X_train_fraud[col].map(freq_encoding)
    X_test_fraud[col] = X_test_fraud[col].map(freq_encoding)

In [21]:
from category_encoders import TargetEncoder

# Target encode 'device_id', 'source', and 'browser'
target_encoder = TargetEncoder(cols=['device_id', 'source', 'browser'])
X_train_fraud = target_encoder.fit_transform(X_train_fraud, y_train_fraud)
X_test_fraud = target_encoder.transform(X_test_fraud)

In [22]:
# Encode only the top N frequent categories and group others
N = 10  # Choose the number of top categories to keep

for col in ['device_id', 'source', 'browser']:
    top_categories = X_train_fraud[col].value_counts().nlargest(N).index
    X_train_fraud[col] = X_train_fraud[col].where(X_train_fraud[col].isin(top_categories), other='Other')
    X_test_fraud[col] = X_test_fraud[col].where(X_test_fraud[col].isin(top_categories), other='Other')

# Then, apply one-hot encoding after reducing categories
X_train_fraud = pd.get_dummies(X_train_fraud, columns=['device_id', 'source', 'browser'], drop_first=True)
X_test_fraud = pd.get_dummies(X_test_fraud, columns=['device_id', 'source', 'browser'], drop_first=True)

Train and evaluate on Fraud Data and Creditcard

In [24]:
# Train and evaluate on Fraud_Data.csv
for model_name, model in models.items():
    train_and_evaluate(model, X_train_fraud, X_test_fraud, y_train_fraud, y_test_fraud, model_name)

# Train and evaluate on creditcard.csv
for model_name, model in models.items():
    train_and_evaluate(model, X_train_cc, X_test_cc, y_train_cc, y_test_cc, model_name)

2024/10/30 12:38:55 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Logistic Regression - Accuracy: 0.9063627039010026, Precision: 1.0, Recall: 0.0, F1: 0.0, ROC-AUC: 0.5


2024/10/30 12:39:07 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Decision Tree - Accuracy: 0.6286602918307249, Precision: 0.15223336371923427, Recall: 0.6491166077738516, F1: 0.24662683761831242, ROC-AUC: 0.6378317679105814


2024/10/30 12:40:10 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

Random Forest - Accuracy: 0.9996137776061234, Precision: 0.9418604651162791, Recall: 0.826530612244898, F1: 0.8804347826086957, ROC-AUC: 0.9132213415754596


2024/10/30 13:01:21 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Gradient Boosting - Accuracy: 0.9983146659176293, Precision: 0.5294117647058824, Recall: 0.1836734693877551, F1: 0.2727272727272727, ROC-AUC: 0.5916960481435117


2024/10/30 13:02:04 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


MLP - Accuracy: 0.9982795547909132, Precision: 0.5, Recall: 0.8163265306122449, F1: 0.6201550387596899, ROC-AUC: 0.9074598325542935


Convolutional Neural Network (CNN)

In [25]:
# Convolutional Neural Network (CNN)
def build_cnn_model(input_shape):
    model = Sequential()
    model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(Flatten())  # Flatten the output from Conv1D before feeding into Dense layers
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # For binary classification
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [28]:
# Ensure y_train_fraud and y_test_fraud are NumPy arrays with the correct dtype
y_train_fraud = np.array(y_train_fraud, dtype='float32')
y_test_fraud = np.array(y_test_fraud, dtype='float32')

# Convert the DataFrame to a NumPy array and reshape it for CNN input
X_train_fraud_reshaped = X_train_fraud.values.reshape(-1, X_train_fraud.shape[1], 1).astype('float32')
X_test_fraud_reshaped = X_test_fraud.values.reshape(-1, X_test_fraud.shape[1], 1).astype('float32')

# Build and train the CNN model
cnn_model = build_cnn_model((X_train_fraud.shape[1], 1))
cnn_model.fit(X_train_fraud_reshaped, y_train_fraud, epochs=10, batch_size=64)

# Make predictions on the test set
y_pred_cnn = cnn_model.predict(X_test_fraud_reshaped)
y_pred_cnn = (y_pred_cnn > 0.5).astype(int)

# Evaluate the model
accuracy, precision, recall, f1, roc_auc = evaluate_model(y_test_fraud, y_pred_cnn)

# Log the model and metrics using MLflow
with mlflow.start_run():
    mlflow.keras.log_model(cnn_model, "CNN")
    log_metrics("CNN", accuracy, precision, recall, f1, roc_auc)


# Print the evaluation results
print(f"CNN - Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1: {f1}, ROC-AUC: {roc_auc}")

/home/brook/Music/10Academy/week-8/Adey-Innovations/env_w8/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-10-30 13:27:47.354071: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Epoch 1/10
1889/1889 ━━━━━━━━━━━━━━━━━━━━ 12s 5ms/step - accuracy: 0.8282 - loss: 2784524.0000
Epoch 2/10
1889/1889 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.8312 - loss: 482568.5312
Epoch 3/10
1889/1889 ━━━━━━━━━━━━━━━━━━━━ 40s 21ms/step - accuracy: 0.8269 - loss: 94172.8438
Epoch 4/10
1889/1889 ━━━━━━━━━━━━━━━━━━━━ 37s 19ms/step - accuracy: 0.8775 - loss: 13.7538
Epoch 5/10
1889/1889 ━━━━━━━━━━━━━━━━━━━━ 36s 19ms/step - accuracy: 0.9067 - loss: 0.3335
Epoch 6/10
1889/1889 ━━━━━━━━━━━━━━━━━━━━ 18s 9ms/step - accuracy: 0.9062 - loss: 0.3123
Epoch 7/10
1889/1889 ━━━━━━━━━━━━━━━━━━━━ 23s 12ms/step - accuracy: 0.9065 - loss: 0.3106
Epoch 8/10
1889/1889 ━━━━━━━━━━━━━━━━━━━━ 25s 13ms/step - accuracy: 0.9057 - loss: 0.3123
Epoch 9/10
1889/1889 ━━━━━━━━━━━━━━━━━━━━ 22s 12ms/step - accuracy: 0.9070 - loss: 0.3093
Epoch 10/10
1889/1889 ━━━━━━━━━━━━━━━━━━━━ 20s 11ms/step - accuracy: 0.9066 - loss: 0.3103
945/945 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step


2024/10/30 13:32:05 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2024/10/30 13:32:24 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


CNN - Accuracy: 0.9063627039010026, Precision: 1.0, Recall: 0.0, F1: 0.0, ROC-AUC: 0.5
